# Data Reading Section:
##### In this section, we will use the following codes, which will be explained below, to read the data we need from the main reference file and convert it to CSV format so that we can use it in the next steps.

---

# Part 1: Importing the required libraries, defining the paths, and creating the required directories if they do not exist.
In this section, we import the libraries and items we need to use them later, and then we define the main paths, such as the main zip file path, the output file directory, and the temp directory, in a relational manner, to be included in the data folder of this project.

In [4]:
import os, zipfile
import pandas as pd
import pyarrow.parquet as pq
from io import BytesIO

# Define paths
main_zip = "..\\data\\raw\\tennis_data.zip"
output_dir = "..\\data\\processed"
temp_dir = "..\\data\\raw\\temp"

os.makedirs(output_dir, exist_ok=True)
os.makedirs(temp_dir, exist_ok=True)

# Part 2: Creating a CSV table generator function from Parquet files
In this section, we have created a very useful function that, based on the keyword of the parquet category name that we give it, goes to the defined path of the main zip file and reads the parquets belonging to the specified tables and the data related to the specified columns from the zip files for each day. In addition to all this, we specify that the records of this table should be unique based on the unique data identifier or that this table can have multiple rows for each unique identifier. Our unique identifier is match_id.

In [ ]:
def build_table(table_keyword, needed_cols, output_name, dedup_on="match_id"):
    """
    table_keyword: like 'event_' or 'home_team_'
    needed_cols: list of needed columns
    output_name: name of output CSV file
    dedup_on: unique column for deduplication (default is 'match_id')
    """
    csv_path = os.path.join(output_dir, output_name)
    if os.path.exists(csv_path):
        os.remove(csv_path)

    all_dfs = []
    row_counter = 0

    with zipfile.ZipFile(main_zip, "r") as main_zip_ref:
        daily_zips = main_zip_ref.namelist()
        print(f"📦 Count of daily zips: {len(daily_zips)}")

        for i, daily_zip_name in enumerate(daily_zips, start=1):
            print(f"🔹 ({i}/{len(daily_zips)}) processing {daily_zip_name} ...")
            main_zip_ref.extract(daily_zip_name, temp_dir)
            daily_zip_path = os.path.join(temp_dir, daily_zip_name)

            with zipfile.ZipFile(daily_zip_path, "r") as daily_zip_ref:
                parquet_files = [f for f in daily_zip_ref.namelist() if f.endswith(".parquet") and table_keyword in f]
                for f in parquet_files:
                    with daily_zip_ref.open(f) as pf:
                        table = pq.read_table(BytesIO(pf.read()))
                        df = table.to_pandas()
                        df = df[[c for c in needed_cols if c in df.columns]]
                        df["date_source"] = daily_zip_name.replace(".zip", "")
                        all_dfs.append(df)
                        row_counter += len(df)

            os.remove(daily_zip_path)

    if all_dfs:
        df_all = pd.concat(all_dfs, ignore_index=True)
        print(f"✅ Shape: {df_all.shape}")
        if dedup_on and dedup_on in df_all.columns:
            df_all = df_all.drop_duplicates(subset=dedup_on)
        else:
            df_all = df_all.drop_duplicates()
        print(f"🧹 after cleaning duplicated rows: {df_all.shape}")

        df_all.to_csv(csv_path, index=False)
        print(f"💾 Saved: {csv_path}")
        print(f"📊 Count of all rows: {len(df_all)}")
    else:
        print(f"⚠️ There is no file for {table_keyword}")

# Part 3: Using the above cell function and creating CSVs of the tables required for analysis according to the columns required from them
In this part, based on the initial analysis we had of the 17 questions in question and the data they required, we extracted a series of tables from a total of 15 tables and a series of their columns that were needed to analyze and answer the 17 questions we needed. Here, we want to extract them from the original raw zip file and convert them to CSV files so that we can use these files later in analyzing and answering the questions.

In [ ]:
build_table(
    table_keyword="event_",
    needed_cols=["match_id", "first_to_serve", "winner_code", "default_period_count", "start_datetime", "match_slug"],
    output_name="event.csv",
    dedup_on="match_id"
)

build_table(
    table_keyword="home_team_",
    needed_cols=["match_id", "player_id", "full_name", "gender", "height", "weight", "plays", "current_rank", "country"],
    output_name="home_team.csv",
    dedup_on="match_id"
)

build_table(
    table_keyword="away_team_",
    needed_cols=["match_id", "player_id", "full_name", "gender", "height", "weight", "plays", "current_rank", "country"],
    output_name="away_team.csv",
    dedup_on="match_id"
)

build_table(
    table_keyword="tournament_",
    needed_cols=["match_id", "tournament_id", "tournament_name", "ground_type", "tennis_points", "start_datetime"],
    output_name="tournament.csv",
    dedup_on="match_id"
)

build_table(
    table_keyword="time_",
    needed_cols=["match_id", "period_1", "period_2", "period_3", "period_4", "period_5", "current_period_start_timestamp"],
    output_name="time.csv",
    dedup_on="match_id"
)

build_table(
    table_keyword="statistics_",
    needed_cols=["match_id", "statistic_name", "home_value", "away_value"],
    output_name="statistics.csv",
    dedup_on=None  # No deduplication because we have multiple rows per match_id in statistics
)

build_table(
    table_keyword="power_",
    needed_cols=["match_id", "set_num", "game_num", "value", "break_occurred"],
    output_name="power.csv",
    dedup_on=None # No deduplication because we have multiple rows per match_id in power
)

build_table(
    table_keyword="pbp_",
    needed_cols=["match_id", "set_id", "game_id", "point_id", "home_point", "away_point"],
    output_name="pbp.csv",
    dedup_on=None # No deduplication because we have multiple rows per match_id in pbp
)